Python libraries used in this notebook.

In [1]:
import pandas as pd
import requests


Generate map of census tracts for the 100th state congress district of Florida

In [2]:
resp = requests.get('https://api.census.gov/data/2020/dec/pl?get=NAME&for=tract%20(or%20part):*&in=state:12%20state%20legislative%20district%20(lower%20chamber):100%20county%20(or%20part):086')
txt = resp.json()
map = pd.DataFrame(txt)

In [3]:
new_header = map.iloc[0] 
map = map[1:] 
map.columns = new_header 
map.reset_index(inplace=True, drop=True)

In [4]:
map['CensusTract'] = map['NAME']

In [5]:
map['CensusTract'] = map['CensusTract'].str.replace("Census Tract ","") 
map['CensusTract'] = map['CensusTract'].str.split(',').str[0]
map['CensusTract'] = map['CensusTract'].str.replace("\(part\)","") 

/tmp/ipykernel_97242/2880120312.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  map['CensusTract'] = map['CensusTract'].str.replace("\(part\)","")


Preview census tract to FL 100th Congressional District map

In [6]:
display(map)

,NAME,state,state legislative district (lower chamber),county (or part),tract (or part),CensusTract
0,"Census Tract 1.07, Miami-Dade County (part), S...",12,100,086,000107,1.07
1,"Census Tract 1.09 (part), Miami-Dade County (p...",12,100,086,000109,1.09
2,"Census Tract 1.15, Miami-Dade County (part), S...",12,100,086,000115,1.15
3,"Census Tract 1.18, Miami-Dade County (part), S...",12,100,086,000118,1.18
4,"Census Tract 1.20, Miami-Dade County (part), S...",12,100,086,000120,1.20
5,"Census Tract 1.21, Miami-Dade County (part), S...",12,100,086,000121,1.21
6,"Census Tract 1.22, Miami-Dade County (part), S...",12,100,086,000122,1.22
7,"Census Tract 1.23, Miami-Dade County (part), S...",12,100,086,000123,1.23
8,"Census Tract 1.25, Miami-Dade County (part), S...",12,100,086,000125,1.25
9,"Census Tract 1.26, Miami-Dade County (part), S...",12,100,086,000126,1.26


Cleanup census age and gender demographics.

In [7]:
df = pd.read_excel(r'/home/jovyan/test_census_data/Input/ACSST5Y2020.S0101-2022-07-11T025947.xlsx')
df['CensusTract'] = df['CensusTract'].str.replace("Census Tract","") 
df['CensusTract'] = df['CensusTract'].str.replace(", Miami-Dade County, Florida","")
df['CensusTract'] = df['CensusTract'].str.strip()
df = df.set_index('CensusTract')

Back fill census tracts.

In [8]:
df.index = pd.Series(df.index).fillna(method='ffill')

Remove aggregate columns from census data.

In [9]:
values = ['Total', 'Percent', 'Percent Female', 'Percent Male']

for i in values:
    df = df[df.AgeGroup != i]

In [10]:
df.rename({'AgeGroup': 'Gender'}, axis=1, inplace=True)

In [11]:
df = df.melt(id_vars=['Gender'], var_name='AgeGroup', value_name='Count', ignore_index=False)

filter age and gender demo by florida 100th census tracts

In [12]:
df = df[df.index.isin(map['CensusTract'])]

Preview final results

In [13]:
display(df)

,Gender,AgeGroup,Count
CensusTract,,,
1.07,Male,Under 5 years,30
1.07,Female,Under 5 years,48
1.15,Male,Under 5 years,143
1.15,Female,Under 5 years,48
1.18,Male,Under 5 years,27
...,...,...,...
1.46,Female,85 years and over,58
38.03,Male,85 years and over,149
38.03,Female,85 years and over,131


In [14]:
df.to_excel('/home/jovyan/test_census_data/Output/output_age_sex_miami_dade_100th.xlsx', sheet_name='AgeSex')